# 5003 project code

#### In this projec, we implemented pair programming through vscode live share. Thus, we tried the findspark to start our spark in vscode below. We shoud install findspark first.

%pip install findspark

In [1]:
%pip install findspark


[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import findspark

findspark.init(spark_home='/Users/zhanghousu/spark/spark-3.3.1-bin-hadoop2',
                python_path='/Users/zhanghousu/conda/bin/python')

from pyspark import SparkContext
sc = SparkContext()

23/04/23 16:07:08 WARN Utils: Your hostname, MacBook-Pro-279.local resolves to a loopback address: 127.0.0.1; using 10.79.140.225 instead (on interface en0)
23/04/23 16:07:08 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/23 16:07:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/04/23 16:07:10 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/04/23 16:07:10 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
23/04/23 16:07:10 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [3]:
# import packages

from pyspark.sql.functions import *
from pyspark.sql import SparkSession

In [4]:
# reading the files in RDD
credits = sc.textFile('data/credits.csv')
links = sc.textFile('data/links.csv')
ratings = sc.textFile('data/ratings.csv')
movies_metadata = sc.textFile('data/movies_metadata.csv')

In [5]:
spark = SparkSession(sc)
cred_df = spark.read.csv(credits, header=True, inferSchema=True, escape='"')
links_df = spark.read.csv(links, header=True, inferSchema=True, escape='"')
ratings_df = spark.read.csv(ratings, header=True, inferSchema=True, escape='"')
movies_metadata_df = spark.read.csv(movies_metadata, header=True, inferSchema=True, escape='"')


23/04/23 16:07:21 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 0 (TID 0): Attempting to kill Python Worker


23/04/23 16:07:28 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 4 (TID 10): Attempting to kill Python Worker


23/04/23 16:07:41 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 6 (TID 33): Attempting to kill Python Worker


## cleanning credit dataset
#### calculating null data

In [6]:
# calucate the null values in each column
null_rows = cred_df.select(*[(sum(when(isnull(c), 1).otherwise(0))).alias(c) for c in cred_df.columns])
null_rows.show()

+----+----+---+
|cast|crew| id|
+----+----+---+
|   0|   0|  0|
+----+----+---+



In [7]:
# drop the rows with null values
cred_df_n = cred_df.dropna()
null_rows_check = cred_df_n.select(*[(sum(when(isnull(c), 1).otherwise(0))).alias(c) for c in cred_df_n.columns])
null_rows_check.show()

+----+----+---+
|cast|crew| id|
+----+----+---+
|   0|   0|  0|
+----+----+---+



## cleanning metadata dataset
#### removing useless features, calculating null data, data StandardScaler

In [8]:
# drop useless columns
mv_meta_df_dp = movies_metadata_df.drop('adult', 'belongs_to_collection', 'homepage', 'imdb_id', 
                                        'original_title', 'overview', 'poster_path', 'release_date', 
                                        'spoken_languages', 'title', 'tagline', 'video')

# show the size of the mv_meta_df_dp
mv_meta_df_dp.count()



45572

In [9]:
# delete the rows with status not equal to "Released"
mv_meta_df_dp = mv_meta_df_dp.filter(col("status") == "Released")
mv_meta_df_dp.count()

44941

In [10]:
# remove status column
mv_meta_df_dp = mv_meta_df_dp.drop('status')
mv_meta_df_dp.show(5)

23/04/23 16:07:51 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 20 (TID 56): Attempting to kill Python Worker
+--------+--------------------+-----+-----------------+----------+--------------------+--------------------+---------+-------+------------+----------+
|  budget|              genres|   id|original_language|popularity|production_companies|production_countries|  revenue|runtime|vote_average|vote_count|
+--------+--------------------+-----+-----------------+----------+--------------------+--------------------+---------+-------+------------+----------+
|30000000|[{'id': 16, 'name...|  862|               en| 21.946943|[{'name': 'Pixar ...|[{'iso_3166_1': '...|373554033|   81.0|         7.7|      5415|
|65000000|[{'id': 12, 'name...| 8844|               en| 17.015539|[{'name': 'TriSta...|[{'iso_3166_1': '...|262797249|  104.0|         6.9|      2413|
|       0|[{'id': 10749, 'n...|15602|               en|   11.7129|[{'name': 'Warner...|[{'iso_3166_1': '...| 

In [11]:
# remove the rows with budget and revenue equal to 0
mv_meta_df_dp = mv_meta_df_dp.filter((col("budget") == 0) & (col("revenue") == 0))
mv_meta_df_dp.count()

34065

In [12]:
mv_meta_df_dp = mv_meta_df_dp.drop('budget', 'revenue')
mv_meta_df_dp.show(5)

23/04/23 16:07:56 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 24 (TID 60): Attempting to kill Python Worker
+--------------------+-----+-----------------+----------+--------------------+--------------------+-------+------------+----------+
|              genres|   id|original_language|popularity|production_companies|production_countries|runtime|vote_average|vote_count|
+--------------------+-----+-----------------+----------+--------------------+--------------------+-------+------------+----------+
|[{'id': 10749, 'n...|15602|               en|   11.7129|[{'name': 'Warner...|[{'iso_3166_1': '...|  101.0|         6.5|        92|
|[{'id': 28, 'name...|45325|               en|  2.561161|[{'name': 'Walt D...|[{'iso_3166_1': '...|   97.0|         5.4|        45|
|[{'id': 35, 'name...|12110|               en|  5.430331|[{'name': 'Columb...|[{'iso_3166_1': '...|   88.0|         5.7|       210|
|[{'id': 18, 'name...| 1710|               en| 10.701801|[{'name': 'Rege

In [13]:
mv_meta_df_dp.printSchema()

root
 |-- genres: string (nullable = true)
 |-- id: string (nullable = true)
 |-- original_language: string (nullable = true)
 |-- popularity: string (nullable = true)
 |-- production_companies: string (nullable = true)
 |-- production_countries: string (nullable = true)
 |-- runtime: string (nullable = true)
 |-- vote_average: string (nullable = true)
 |-- vote_count: string (nullable = true)



In [14]:
from pyspark.sql.types import DoubleType

# create DoubleType list
columns_to_convert = ["popularity", "runtime", "vote_average", "vote_count"]

# convert the columns to DoubleType
for column in columns_to_convert:
    mv_meta_df_dp = mv_meta_df_dp.withColumn(column, mv_meta_df_dp[column].cast(DoubleType()))

# check the schema
mv_meta_df_dp.printSchema()

root
 |-- genres: string (nullable = true)
 |-- id: string (nullable = true)
 |-- original_language: string (nullable = true)
 |-- popularity: double (nullable = true)
 |-- production_companies: string (nullable = true)
 |-- production_countries: string (nullable = true)
 |-- runtime: double (nullable = true)
 |-- vote_average: double (nullable = true)
 |-- vote_count: double (nullable = true)



In [17]:
from pyspark.sql.functions import mean, stddev, col

# create a list of columns to normalize
for column in columns_to_convert:
    # calculate the mean and standard deviation
    mean_value, stddev_value = mv_meta_df_dp.select(mean(col(column)), stddev(col(column))).first()

    # normalize the column
    mv_meta_df_dp = mv_meta_df_dp.withColumn(column, (col(column) - mean_value) / stddev_value)

mv_meta_df_dp.show(5)


23/04/23 16:14:55 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 50 (TID 86): Attempting to kill Python Worker
+--------------------+-----+-----------------+-------------------+--------------------+--------------------+--------------------+-------------------+------------------+
|              genres|   id|original_language|         popularity|production_companies|production_countries|             runtime|       vote_average|        vote_count|
+--------------------+-----+-----------------+-------------------+--------------------+--------------------+--------------------+-------------------+------------------+
|[{'id': 10749, 'n...|15602|               en|  4.206316413101168|[{'name': 'Warner...|[{'iso_3166_1': '...| 0.24766515838470915| 0.4855626862661089|1.5891692288003527|
|[{'id': 28, 'name...|45325|               en|0.37941274243353634|[{'name': 'Walt D...|[{'iso_3166_1': '...|  0.1497112793968787|-0.0461136633509578|0.5691413503944264|
|[{'id': 35, 'name.

In [18]:
def helper(input_str):
    return input_str.replace("None", "'None'")

helper_udf = udf(helper)

new_df = cred_df.withColumn("cast_new", helper_udf("cast"))
new_df.show(10)

23/04/23 16:15:00 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 51 (TID 87): Attempting to kill Python Worker
+--------------------+--------------------+-----+--------------------+
|                cast|                crew|   id|            cast_new|
+--------------------+--------------------+-----+--------------------+
|[{'cast_id': 14, ...|[{'credit_id': '5...|  862|[{'cast_id': 14, ...|
|[{'cast_id': 1, '...|[{'credit_id': '5...| 8844|[{'cast_id': 1, '...|
|[{'cast_id': 2, '...|[{'credit_id': '5...|15602|[{'cast_id': 2, '...|
|[{'cast_id': 1, '...|[{'credit_id': '5...|31357|[{'cast_id': 1, '...|
|[{'cast_id': 1, '...|[{'credit_id': '5...|11862|[{'cast_id': 1, '...|
|[{'cast_id': 25, ...|[{'credit_id': '5...|  949|[{'cast_id': 25, ...|
|[{'cast_id': 1, '...|[{'credit_id': '5...|11860|[{'cast_id': 1, '...|
|[{'cast_id': 2, '...|[{'credit_id': '5...|45325|[{'cast_id': 2, '...|
|[{'cast_id': 1, '...|[{'credit_id': '5...| 9091|[{'cast_id': 1, '...|
|[{'cast_id'

In [27]:
from pyspark.sql.functions import from_json, col, explode, get_json_object, array
from pyspark.sql.types import StructType, StructField,\
    StringType, IntegerType, ArrayType

json_schema = StructType([
    StructField("name", StringType(), True),
])

# Parse the JSON array and add it as a new column 'cast_array'
new_df1 = new_df.withColumn("cast_array", from_json(col("cast_new"), ArrayType(json_schema)))

new_df1.show()

23/04/23 16:26:08 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 57 (TID 93): Attempting to kill Python Worker
+--------------------+--------------------+-----+--------------------+--------------------+
|                cast|                crew|   id|            cast_new|          cast_array|
+--------------------+--------------------+-----+--------------------+--------------------+
|[{'cast_id': 14, ...|[{'credit_id': '5...|  862|[{'cast_id': 14, ...|[{Tom Hanks}, {Ti...|
|[{'cast_id': 1, '...|[{'credit_id': '5...| 8844|[{'cast_id': 1, '...|[{Robin Williams}...|
|[{'cast_id': 2, '...|[{'credit_id': '5...|15602|[{'cast_id': 2, '...|[{Walter Matthau}...|
|[{'cast_id': 1, '...|[{'credit_id': '5...|31357|[{'cast_id': 1, '...|[{Whitney Houston...|
|[{'cast_id': 1, '...|[{'credit_id': '5...|11862|[{'cast_id': 1, '...|[{Steve Martin}, ...|
|[{'cast_id': 25, ...|[{'credit_id': '5...|  949|[{'cast_id': 25, ...|[{Al Pacino}, {Ro...|
|[{'cast_id': 1, '...|[{'credit_id':

In [29]:
casts = new_df1.select(explode(col("cast_array")["name"]).alias("cast"),"id")
casts.show(20)

23/04/23 16:26:42 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 58 (TID 94): Attempting to kill Python Worker
+-----------------+----+
|             cast|  id|
+-----------------+----+
|        Tom Hanks| 862|
|        Tim Allen| 862|
|      Don Rickles| 862|
|       Jim Varney| 862|
|    Wallace Shawn| 862|
|John Ratzenberger| 862|
|      Annie Potts| 862|
|      John Morris| 862|
|  Erik von Detten| 862|
|   Laurie Metcalf| 862|
|     R. Lee Ermey| 862|
|    Sarah Freeman| 862|
|    Penn Jillette| 862|
|   Robin Williams|8844|
|    Jonathan Hyde|8844|
|    Kirsten Dunst|8844|
|   Bradley Pierce|8844|
|      Bonnie Hunt|8844|
|    Bebe Neuwirth|8844|
| David Alan Grier|8844|
+-----------------+----+
only showing top 20 rows



In [31]:
distinct_categories_count = mv_meta_df_dp.select("genres").distinct().count()

# 输出结果
print("Distinct categories count:", distinct_categories_count)

Distinct categories count: 3198


In [ ]:

# spark.stop()